In [3]:
from langchain.llms import OpenAI

In [4]:
import os
os.environ['OPENAI_API_KEY'] = ""

In [6]:
llm = OpenAI(openai_api_key = os.environ['OPENAI_API_KEY'], temperature=0.6)

In [7]:
text = "What is the capital of Turkey"
print(llm.predict(text))



The capital of Turkey is Ankara.


In [8]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ""

In [9]:
from langchain import HuggingFaceHub

In [25]:
llm_hugginggace = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0, "model_length":64})

In [28]:
output = llm_hugginggace.predict("Can you tell me the capital of Russia?")

In [29]:
print(output)

moscow


In [30]:
output = llm_hugginggace.predict('Can you write a poem about computers')

In [31]:
print(output)

i love my computer i love my computer i love my computer i love my


### Prompt Templates and Chains

In [32]:
from langchain.prompts import PromptTemplate

In [33]:
prompt_template = PromptTemplate(input_variables=["country"],
                                 template="Tell me the capital of {country}")

In [34]:
prompt_template.format(country="Turkey")

'Tell me the capital of Turkey'

In [37]:
from langchain.chains import LLMChain

In [38]:
chain=LLMChain(llm=llm, prompt=prompt_template)
chain.run("Turkey")

'\n\nThe capital of Turkey is Ankara.'

### Combining Multiple Chains Using Simple Sequential Chain

In [39]:
capital_template = PromptTemplate(input_variables=['country'],
                                template="Please tell me the capital of the {country}")

In [41]:
capital_chain = LLMChain(llm=llm, prompt=capital_template)

In [42]:
famous_template = PromptTemplate(input_variables=['capital'],
                                 template="Suggest some amazing places to visit in {capital}")

In [43]:
famous_chain = LLMChain(llm=llm, prompt=famous_template)

In [44]:
from langchain.chains import SimpleSequentialChain

In [45]:
chain=SimpleSequentialChain(chains=[capital_chain, famous_chain])
chain.run("Turkey")

" Some amazing places to visit in Ankara are:\n\n1. Anıtkabir: This is the mausoleum of Mustafa Kemal Atatürk, the founder of modern Turkey. It is a grand and impressive structure, with a museum and gardens surrounding it.\n\n2. Kızılay Square: This is the main square of Ankara, and is always bustling with activity. It is a popular spot for locals and tourists alike, with many shops, restaurants, and cafes to explore.\n\n3. Ankara Citadel: This ancient fortress is a must-see for history buffs. It offers stunning views of the city and is a great place to learn about Ankara's past.\n\n4. Museum of Anatolian Civilizations: This museum houses a vast collection of artifacts from various ancient civilizations that once inhabited the region. It is a great place to learn about the rich history of Anatolia.\n\n5. Atakule Tower: This 410-foot tall tower offers panoramic views of the city and is a popular spot for tourists. It also has a shopping mall and restaurants inside.\n\n6. Gençlik Park: T

### Sequential Chain

In [51]:
capital_template = PromptTemplate(input_variables=['country'],
                                template="Please tell me the capital of the {country}")
capital_chain = LLMChain(llm=llm, prompt=capital_template, output_key="capital")

In [52]:
famous_template = PromptTemplate(input_variables=['capital'],
                                 template="Suggest some amazing places to visit in {capital}")
famous_chain = LLMChain(llm=llm, prompt=famous_template, output_key="places")

In [53]:
from langchain.chains import SequentialChain

In [54]:
chain = SequentialChain(chains=[capital_chain, famous_chain],
                        input_variables=['country'],
                        output_variables=['capital', "places"])

In [55]:
chain({'country':"Turkey"})

{'country': 'Turkey',
 'capital': '\n\nThe capital of Turkey is Ankara.',
 'places': " Some amazing places to visit in Ankara are:\n\n1. Anitkabir (Ataturk Mausoleum) - This is the final resting place of Mustafa Kemal Ataturk, the founder of modern Turkey. The mausoleum is a grand and impressive structure, with a museum dedicated to Ataturk's life and achievements.\n\n2. Ankara Citadel - This ancient fortress sits on a hill overlooking the city and offers stunning views of Ankara. It has been standing since the 3rd century BC and is a great place to explore and learn about the city's history.\n\n3. Kizilay Square - This is the main square of Ankara and a popular gathering place for locals. It is surrounded by shops, restaurants, and cafes, and is a great spot to people-watch and soak in the city's atmosphere.\n\n4. Museum of Anatolian Civilizations - This museum showcases the rich history of Anatolia, with artifacts dating back to the Paleolithic era. It is a must-visit for history buf

### ChatModels with ChatOpenAI

In [56]:
from langchain.chat_models import ChatOpenAI

In [57]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

In [58]:
chatllm = ChatOpenAI(openai_api_key = os.environ['OPENAI_API_KEY'], temperature=0.6, model='gpt-3.5-turbo')

In [61]:
chatllm([SystemMessage(content='You are a stand-up comedian AI assistant'),
 HumanMessage(content='Give me some jokes about AI')])

AIMessage(content='Sure, here are a few AI jokes for you:\n\n1. Why did the AI go on a diet? It had too many bytes!\n\n2. What did the AI say to the computer programmer? "I think you\'re coding me crazy!"\n\n3. Why did the AI cross the road? To optimize its path-finding algorithm!\n\n4. Why was the AI always confident? It had a strong neural network!\n\n5. How do AI robots greet each other? "Hello, world!"\n\n6. Why did the AI fail its driving test? It couldn\'t keep its lanes of code!\n\n7. What do you call an AI that loves to dance? An Algorhythm!\n\n8. Why did the AI become a stand-up comedian? It had a great sense of humor - artificial intelligence!\n\nRemember, these jokes are all in good fun!')

### Prompt Template + LLM + Output Parser

In [62]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [63]:
class CommaSeparatedOutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(',')

In [66]:
template = "You are a helpful assistant. When the user gives any input to you, you should generate five words synonyms in a comma separated list."

In [67]:
human_template = '{text}'

In [68]:
chatprompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

In [69]:
chain = chatprompt|chatllm|CommaSeparatedOutput()

In [70]:
chain.invoke({"text":"intelligent"})

['smart', ' clever', ' brilliant', ' knowledgeable', ' sharp']

In [71]:
chain_without_parser = chatprompt|chatllm
chain_without_parser.invoke({"text":"intelligent"})

AIMessage(content='smart, clever, bright, brilliant, knowledgeable')